In [3]:
import argparse
import os
import subprocess
import xarray as xr
import pyrte.pyrte as pyrte
import sys

rte_rrtmgp_dir = "/Users/josue/Documents/makepath/rte-python/rrtmgp-data"
clear_sky_example_files = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/inputs"

conds_file = xr.load_dataset(
    f"{clear_sky_example_files}/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)
# lw_gas_coeffs = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-lw-g256.nc")
sw_gas_coeffs = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-gas-sw-g224.nc")


rfmip = conds_file
kdist = sw_gas_coeffs

flxdn_file = xr.load_dataset(
    f"{clear_sky_example_files}/rsd_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
)
flxup_file = xr.load_dataset(
    f"{clear_sky_example_files}/rsu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
)


# RRTMGP won't run with pressure less than its minimum. so we add a small value to the minimum pressure
press_min = rfmip["pres_level"].min()
min_index = rfmip["pres_level"].argmin()
rfmip["pres_level"][min_index] = press_min + sys.float_info.epsilon

# Filter angles < 90, as it should be nighttime
rfmip["solar_zenith_angle"] = rfmip["solar_zenith_angle"].where(rfmip["solar_zenith_angle"] <= 90)

# use mo_gas_optics_rrtmgp,  only: ty_gas_optics_rrtmgp

# ! Compute the optical properties of the atmosphere and the Planck source functions
# !    from pressures, temperatures, and gas concentrations...
# !
# call stop_on_err(k_dist%gas_optics(p_lay(:,:,b), &
#                                     p_lev(:,:,b),       &
#                                     t_lay(:,:,b),       &
#                                     gas_conc_array(b),  &
#                                     optical_props,      &
#                                     toa_flux))
